In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import seaborn as sns
import itertools
from itertools import groupby
import os
import re
from pathlib import Path
import ast

In [2]:
def assign_flags(gaze_df, events):
    """
    Given a gaze DataFrame and event intervals, assign flag and emotion_side.
    Returns the DataFrame with added 'flag' and 'emotion_side' columns.
    """
    # Initialize new columns.
    gaze_df['flag'] = -1
    gaze_df['emotion_side'] = "N"  # Default to "N" if no emotion side is applicable.
    
    # Apply flag 0 for fixation intervals.
    for start, stop in events['fixation']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 0
    
    # Apply flag 1 for faces stimuli intervals and assign emotion.
    for start, stop, emotion in events['faces']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 1
        gaze_df.loc[mask, 'emotion_side'] = emotion  # "L", "R", or "N" if not captured.
    
    # Apply flag 2 for pause intervals.
    for start, stop in events['pause']:
        mask = (gaze_df['Time'] >= float(start)) & (gaze_df['Time'] <= float(stop))
        gaze_df.loc[mask, 'flag'] = 2
    
    return gaze_df

In [ ]:
pathlist = Path('../../asc_data').rglob('*.asc')
task_data_df = []
df_final = pd.DataFrame()
time_delay_non_cued = []
placement_position_saccade = []
# for all participant
for path in pathlist:
    participant= re.search(r"(P\d+)", os.path.basename(str(path.stem))).group(1)
    # if participant == 'P157':
    f = open(str(path), 'r') # 'r' = read
    lines = f.read()
    f.close()
    # j = 4
    # n = 30
    indices = []
    current_start_index = 0
    not_list = []
    # for i in range(1):
    index_init1 = re.search(r'Dot Probe Faces Experiemnt Starts',lines[current_start_index:-1]).span()[1]
    index_init2 = re.search(r'\n',lines[current_start_index + index_init1:-1]).span()[1]
    index_init =  current_start_index + index_init1 + index_init2

    index_finit1 = re.search(r'Dot Probe Faces Experiment Ends',lines[index_init:-1]).span()[0]
    index_finit = index_init + index_finit1

    indices.append([index_init, index_finit-1])
    current_start_index = index_finit

    # if(i + 1) % 5 == 0:
    #     j = int(j/2)

    new_vec = lines[index_init: index_finit-1].split("\n")
    new_new_vec = []
    #0 = blink, 1 fixation, 2 saccade
    stateL = 0
    stateR = 0
    
    events = {
        # 'experiment': None,
        'fixation': [],
        'faces': [],
        'pause': []
    }
    fixation_start = None
    faces_start = None
    faces_emotion = None
    pause_start = None
    exp_start = None
    exp_end = None

    # print('....................',len(new_vec))
    for k in range(1,len(new_vec)):
        la = new_vec[k].split("\t")
        if len(la) == 1:
            if 'SSACC L' in la[0]:
                stateL = 2
            if 'SFIX L' in la[0]:
                stateL = 1
            if 'SBLINK L' in la[0]:
                stateL = 0
            if 'SSACC R' in la[0]:
                stateR = 2
            if 'SFIX R' in la[0]:
                stateR = 1
            if 'SBLINK R' in la[0]:
                stateR = 0

        if len(la) == 2:
            ts = la[1].split(' ')[0]
            if 'Fixation_Cross_Start' in la[1]:
                fixation_start = ts
            elif 'Fixation_Cross_Stop' in la[1] and fixation_start is not None:
                events['fixation'].append((fixation_start, ts))
                fixation_start = None

            if 'Faces_Stimuli_Start' in la[1]:
                faces_start = ts
                faces_emotion = None
            
            if 'Emotion_Side' in la[1]:
                m = re.search(r"emotion_side:\s*(\w+)", la[1], re.IGNORECASE)
                if m:
                    emo = m.group(1).lower()
                    faces_emotion = "L" if emo == "left" else "R"

            elif 'Faces_Stimuli_Stop' in la[1] and faces_start is not None: 
                if faces_emotion is None:
                    faces_emotion = "N"
                events['faces'].append((faces_start, ts, faces_emotion))
                faces_start = None
                faces_emotion = None

            if 'Pause_Start' in la[1]:
                pause_start = ts
            elif 'Pause_Stop' in la[1] and pause_start is not None:
                events['pause'].append((pause_start, ts))
                pause_start = None

        if len(la) == 9:
            if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                continue
            else:
                la[-1] = stateL
                la.append(stateR)
                new_new_vec.append(la)
        if len(la) == 6:
            if any(marker in la[0] for marker in ['ESACC L', 'EFIX L', 'EBLINK L', 'ESACC R', 'EFIX R', 'EBLINK R']):
                continue
            else:
                la[-1] = stateL
                la.append(stateR)
                new_new_vec.append(la)

    with open("new_new_vec.txt", "w") as file:
        for item in new_new_vec:
            file.write(str(item) + "\n")

    # print(events)

    ls = [[np.nan if x == '   .' else x for x in y] for y in new_new_vec]
    if len(ls)>0:
        ls = np.swapaxes(ls,0,1)
        current_trial = np.asarray(ls, dtype=float)
        # if participant =='P205':
        # print(len(current_trial))
        if len(current_trial) == 10:
            my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[4][:],current_trial[5][:],current_trial[-1][:]]),0,1)), columns=['Time','RX','RY','state'])
        else:
            # print(current_trial[7][:])
            my_df = pd.DataFrame(list(np.swapaxes(np.array([current_trial[0][:],current_trial[1][:],current_trial[2][:],current_trial[5][:]]),0,1)), columns=['Time','RX','RY','state'])
        my_df['participants'] = participant
        assign_flags(my_df, events)

        task_data_df.append(my_df)
        df_final = pd.concat([df_final,my_df], ignore_index=True)
           

                
        

df_final.to_csv("../../processed_data/processed_data_asc_task_faces_all.csv", index=False)
    

In [4]:
df_final

,Time,RX,RY,state,participants,flag,emotion_side
0,84061278.0,849.5,509.5,0.0,P004,-1,N
1,84061279.0,849.9,510.9,0.0,P004,-1,N
2,84061280.0,849.5,510.8,0.0,P004,-1,N
3,84061281.0,849.2,509.4,0.0,P004,0,N
4,84061282.0,849.7,508.9,0.0,P004,0,N
...,...,...,...,...,...,...,...
2867978,428031740.0,883.8,544.0,1.0,P210,1,R
2867979,428031741.0,885.1,544.0,1.0,P210,1,R
2867980,428031742.0,885.1,545.9,1.0,P210,1,R
2867981,428031743.0,883.8,547.7,1.0,P210,1,R


In [46]:
df_final.tail()

,Time,RX,RY,state,participants,flag,emotion_side
68266,71324749.0,1467.0,547.8,1.0,P157,1,L
68267,71324750.0,1467.0,548.0,1.0,P157,1,L
68268,71324751.0,1465.5,549.7,1.0,P157,1,L
68269,71324752.0,1463.6,551.3,1.0,P157,1,L
68270,71324753.0,1463.4,552.5,1.0,P157,2,L


In [40]:
df_final[df_final['flag']==-1]

,Time,RX,RY,state,participants,flag,emotion_side
0,71256483.0,951.3,526.3,0.0,P157,-1,N
1,71256484.0,952.5,524.6,0.0,P157,-1,N
2,71256485.0,951.5,526.3,0.0,P157,-1,N
3,71256486.0,949.9,528.8,0.0,P157,-1,N
